In [14]:
from glob import glob
import pandas as pd
from sqlite3 import connect
import os
from astropy.io import fits
from bokeh.io import output_notebook, output_file
from bokeh.layouts import column
from bokeh.plotting import figure, show, output_file, save
from bokeh.models import Label
from bokeh.models.tools import BoxZoomTool, PanTool, ResetTool
import numpy as np

conn = connect("../django/QuadEBdb/db.sqlite3")

spectra = []
for path in glob("../Files/2023??/pepsi*.*"):
    filename = os.path.basename(path)
    target, _, _, _, cd, arm, exposure, _, datetime, _ = conn.execute(
        """
        select t.local_id, srd.*
        from tom_spectrumrawdata srd
        join tom_target t on t.id = srd.target_id
        where uri = ?;
        """,
        (filename,),
    ).fetchone()
    spectra.append((target, cd, arm, exposure, datetime, path, filename))
spectra.sort(key=lambda x: x[4])

# output_notebook()
output_file("combined spectra.html")
plots = []
for target, cd, arm, exposure, datetime, path, filename in spectra:
    data = fits.getdata(path)
    quality = conn.execute(
        """
        select se.quality
        from tom_spectrumrawdata srd
        join tom_spectrumevaluation se on se.spectrum_id = srd.id
        where srd.uri=?;
        """,
        (filename,),
    ).fetchone()

    p = figure(
        title=f"{target:<14s}: {arm:>4s}, {cd:<14s}, {exposure:>4.0f}sec, {datetime[:19]}  File: {filename:<30s}",
        sizing_mode="stretch_width",
        tools = [BoxZoomTool(), PanTool(), ResetTool()],
    )
    p.line(data["Arg"], np.clip(data["Fun"], a_min=None, a_max=1.3), line_width=0.2)
    p.xaxis.ticker.num_minor_ticks = 25
    p.add_layout(Label(x=np.min(data["Arg"]), y=1.2, x_units="data", y_units="data", text=f"Quality={quality}"))

    # output_file(filename=f"{target}_{arm}.html", title=p.title)
    f = f"{target}_{arm[:1]}_{datetime[:13]}h{datetime[14:16]}m{datetime[17:19]}s.html"
    output_file(filename=f, title=p.title)
    plots.append(p)
    # save(p)

save(column(plots, sizing_mode="scale_width"))


'c:\\Users\\User\\Dropbox\\School\\Astronomy\\QuadEB\\Observation Planning\\TIC 336882813_B_2023-12-06 08h26m24s.html'

: 

In [4]:
quality = conn.execute(
        """
        select se.quality
        from tom_spectrumrawdata srd
        join tom_spectrumevaluation se on se.spectrum_id = srd.id
        where srd.uri=?;
        """,
        (filename,),
    ).fetchone()

In [1]:
from astropy.table import Table

t = Table.read('../django/QuadEBdb/Data Files/SpectrumEvaluations.csv', format="ascii.csv")
t

Target,Datetime_utc,Rating
str13,str24,float64
TIC 204698586,2023-05-01T04:10:47.945Z,1.0
TIC 161043618,2023-05-01T02:51:34.100Z,1.0
TIC 123098844,2023-05-01T14:23:26.864Z,2.0
TIC 89278612,2023-05-01T12:47:20.368Z,2.0
TIC 278352276,2023-05-01T22:33:43.886Z,3.0
TIC 322727163,2023-05-02T10:04:49.666Z,3.0
TIC 375325607,2023-05-02T10:22:49.471Z,1.0
TIC 123098844,2023-06-14T10:56:41.427Z,1.0
TIC 89278612,2023-06-14T10:52:55.719Z,1.0
